# Status #

- [x] Outline
- [ ] Introduction
- [ ] Exercise 1
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 2
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 3
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Exercise 4
  - [ ] Code
  - [ ] Discussion
  - [ ] Checking
- [ ] Conclusion

# Introduction #

Run the cell below to set everything up.

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex3 import *

### 1) Apply Pooling

In [ ]:
# YOUR CODE HERE
image_condense = ____

q_4.check()

In [ ]:
#%%RM_IF(PROD)%%
q_4.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_4.hint()
#_COMMENT_IF(PROD)_
q_4.solution()

Run the next cell to see what maximum pooling did to the feature!

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_detect)
)
plt.axis('off')
plt.show();

### 2) Explore Invariance

This next code cell will randomly apply a small shift to the circle and then apply maximum pooling several times. Run this cell a few times and make note of the feature produced at the end.

If you were wanting to teach a classifier to recognize circles, would it be helpful to include all of these "shifted" circles in your dataset?

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_4.hint()
#_COMMENT_IF(PROD)_
q_4.solution()

In Lesson 6, we'll explore this idea more when we learn about **Data Augmentation**.

### 3) What about Average Pooling?

In this exercise, we'll look at an alternative to maximum pooling that was once very popular: **average pooling**.

Run the following cell to see the effect of average pooling.

What happened? Can you see why average pooling isn't so popular anymore?

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_3.hint()
#_COMMENT_IF(PROD)_
q_3.solution()

### 4) What about Global Average Pooling?

We saw in the previous exercise that average pooling doesn't work very well as an alternative to maximum pooling. There is, however, one kind of average pooling that *does* work well: **global average pooling**. A `GlobalAvgPool2D` layer is often used as an alternative to the hidden `Dense` layers in the head of the network, like so:

<!--md-->

In [ ]:
model = keras.Sequential([
    pretrained_base,
    layers.GlobalAvgPool2D(),
    layers.Dense(1, activation='sigmoid'),
])

<!--end_md-->

What is this layer doing? Notice that we no longer have the `Flatten` layer that usually comes after the base to transform the 2D feature data to 1D data needed by the classifier. The `GlobalAvgPool2D` layer is serving this function instead, but instead of "unstacking" the feature (like `Flatten`), it simply replaces the entire feature with its average value. This works suprisingly well, and has the advantage of reducing the number of parameters in the model.

First run the following cell to see what `GlobalAvgPool2D` does to a stack of features.

Now rewrite this model to use a `GlobalAvgPool2D` layer instead of the `Flatten` and `Dense` layers.

Lastly, let's train the model. Run the following cell for credit on this exercise.

# Conclusion #
